本文学习于《Effective Python:编写高质量Python代码的59个有效方法》，作者:布雷特.斯拉特金;整理:Weiyang,微信号:damo894127201,Time:2019.02.02

# 尽量用异常来表示特殊情况，而不要返回None

1. 用None这个返回值来表示特殊意义的函数，很容易使调用者犯错，因为None和0及空字符串之类的值，在条件表达式里都会被评估为False
2. 函数在遇到特殊情况时，应该抛出异常，而不要返回None。调用者看到该函数的文档中所描述的异常之后，应该就会编写相应的代码来处理它们了

```Python
x,y = 5,2
try:
    result = divide(x,y)
except ValueError:
    print('Invalid inputs')
else:
    print('Result is %.f' % result)
```

调用者无需用条件语句来判断函数的返回值，因为如果函数没有抛出异常，返回值自然就是正确的。这样写出来的异常处理代码，也比较清晰。

# 了解如何在闭包里使用外围作用域中的变量

Python支持闭包(closure)，闭包是一种定义在某个作用域中的函数，这种函数引用了那个作用域里面的变量。

Python的函数是一级对象(first-class object)，也就是说，我们可以直接引用函数，把函数赋给变量，把函数当成参数传给其它函数，并通过表达式及if语句对其进行比较和判断。

1. 对于定义在某作用域内的闭包来说，它可以引用这些作用域中的变量
2. 使用默认方式对闭包内的变量赋值，不会影响外围作用域中的同名变量
3. 在Python3中，程序可以在闭包内用nonlocal语句来修饰某个名称，使该闭包能够修改外围作用域中的同名变量
4. 在Python2中，程序可以使用可变值(例如，包含单个元素的列表)来实现与nonlocal语句相仿的机制
5. 除了那种比较简单的函数，尽量不要使用nonlocal语句

在表达式中引用变量时，Python解释器将按如下顺序遍历各作用域，以解析该引用：
1. 当前函数的作用域
2. 任何外围作用域(例如，包含当前函数的其他函数)
3. 包含当前代码的那个模块的作用域(也叫全局作用域，global scope)
4. 内置作用域(也就是包含len及str等函数的那个作用域)

如果上面这些地方都没有定义过名称相符的变量，那就抛出NameError异常

给变量赋值时，规则有所不同。如果当前作用域内已经定义了这个变量，那么该变量就会具备新值。若是当前作用域内没有这个变量，Python则会把这次赋值视为对该变量的定义。而新定义的这个变量，其作用域也就是包含赋值操作的这个函数。

Python3获取闭包内的数据：nonlocal语句

```Python
def sort_priority(numbers,group):
    found = False
    def helper(x):
        nonlocal found // nonlocal语句,found是变量
        if x in group:
            found = True
            return (1,x)
    numbers.sort(key=helper)
    return found
```

用**nonlocal**语句来表面这样的意图：给相关变量(这里是found)赋值的时候，应该在上层作用域中查找该变量。nonlocal的唯一限制在于，它不能延伸到**模块级别**，这是为了防止它污染全局作用域。

**nonlocal语句**清楚地表明：如果在闭包内给该变量赋值，那么修改的其实是**闭包外**那个作用域中的变量。这与global语句互为补充，**global**用来表示对该变量的赋值操作，将会直接修改**模块作用域**里的那个变量。

# 考虑用生成器来改写直接返回列表的函数

生成器(generator)是使用**yield**表达式的函数。调用生成器函数时，它并不会真的运行，而是会返回迭代器。每次在这个迭代器上面调用内置的**next()**函数时,迭代器会把生成器推进到下一个**yield**表达式那里。生成器传给**yield**的每一个值，都会由迭代器返回给调用者。

例如，我们要查出字符串中每个单词的首字母，在整个字符串里的位置。

下面这段代码，用append方法将这些词的首字母索引添加到result列表中，并在函数结束时将其返回给调用者。

In [1]:
def index_words(text):
    result = []
    if text:
        result.append(0)
    for index,letter in enumerate(text):
        if letter == ' ':
            result.append(index+1)
    return result

address = 'Four score and seven years age...'
result = index_words(address)
print(result[:3])

[0, 5, 11]


下面用生成器函数，来改造一下

In [2]:
def index_words_iter(text):
    if text:
        yield 0
    for index,letter in enumerate(text):
        if letter == ' ':
            yield index + 1

address = 'Four score and seven years age...'
result = index_words_iter(address)
result

<generator object index_words_iter at 0x7f7f2dc10780>

调用生成器后所返回的迭代器，可以传递给内置的list函数，以将其转换为列表。或者直接for循环迭代出来。

In [3]:
list(result)

[0, 5, 11, 15, 21, 27]

这里需要注意的是，函数返回的那个迭代器，是有状态的，调用者不能反复使用它。例如，再使用一次，结果如下

In [4]:
list(result)

[]

In [5]:
address = 'Four score and seven years age...'
result = index_words_iter(address)
for i in result:
    print('for: ',i)

for:  0
for:  5
for:  11
for:  15
for:  21
for:  27


这个函数不需要包含与result列表相交互的那些代码，因而看起来比较清晰。同时，之前result列表那个函数要先把所有结果都放在列表里面，如果输入量非常大，那么程序就有可能耗尽内存资源。相反，用生成器改造的函数，可以应对任意长度的输入数据。

1. 使用生成器比把收集到的结果放入列表返回给调用者更加清晰
2. 由生成器函数所返回的那个迭代器，可以把生成器函数体中，传给yield表达式的那些值，逐次产生出来
3. 无论输入量有多大，生成器都可能产生一系列输出，因为这些输入量和输出量，都不会影响它在执行时所耗的内存

# 在参数上面迭代时，要多加小心

1. 函数在输入的参数上面多次迭代时要当心：如果参数是迭代器，那么可能会导致奇怪的行为并错失某些值
2. Python的迭代器协议，描述了容器和迭代器应该如何与iter和next内置函数,for循环及相关表达式相互配合
3. 把__iter__方法实现为生成器，即可定义自己的容器类型
4. 想判断某个值是迭代器还是容器，可以拿该值为参数，两次调用iter函数，若结果相同，则是迭代器，调用内置的next函数，即可令该迭代器前进一步

迭代器协议有这样的约定：如果把迭代器对象传给内置的iter函数，那么此函数会把该迭代器返回；反之，如果传给iter函数的是个容器类型的对象，那么iter函数则每次都返回新的迭代器对象。故而，可以根据iter函数的这种行为来判断输入值是不是迭代器对象，如果是则抛出TypeError。

定义一个容器类container，从文件中读取数据:需实现__iter__方法

```Python
class ReadVisits(object):
    def __init__(self,data_path):
        self.data_path = data_path
        
    def __iter__(self):
        with open(self.data_path) as f:
            for line in f:
                yield line
```

判断传进来的参数是否是迭代器

```Python
def is_iter(lst):
    if iter(lst) is iter(lst):
        raise TypeError('Must supply a container')
```

# 用数量可变的位置参数减少视觉杂讯

令函数接受可选的位置参数，能够使代码更加清晰，并能减少视觉杂讯(visual noise)。位置参数习惯上用\*args，又称star args,星号参数。

1. 在def语句中使用\*args，即可令函数接受数量可变的位置参数
2. 调用函数时，可以采用\*操作符，把序列中的元素当成位置参数，传给该函数
3. 在生成器使用\*操作符时，可能导致程序耗尽内存并崩溃
4. 在已经接受\*args参数的函数上面继续添加位置参数，可能会产生难以排查的bug

在生成器使用\*操作符，可能导致程序耗尽内存并崩溃。这是因为，变长参数在传给函数时，总是要先转化成元组。这意味着，如果用带有\*操作符的生成器为参数，来调用这种函数，那么Python就必须先把该生成器完整地迭代一轮，并把生成器所生成的每一个值，都放入元组中。

# 用关键字参数来表达可选的行为

1. 函数参数可以按位置或关键字来指定
2. 只使用位置参数来调用函数，可能会导致这些参数值的含义不够明确，而关键字参数则能够阐明每个参数的意图
3. 给函数添加新的行为时，可以使用带默认值的关键字参数，以便与原有的函数调用代码保持兼容
4. 可选的关键字参数，总是应该以关键字形式来指定，而不应该以位置参数的形式来指定
5. 对于接受\*arg的函数，如果要在扩充其参数的时候，与已有的那些函数调用代码保持兼容，那么就应该把新参数定义为可选的关键字参数。但是还有一种更好的方法，就是采用只能通过关键字形式来指定的参数
6. 关键字参数的顺序不限定
7. 位置参数如果和关键字参数同时使用，位置参数必须出现在关键字参数之前

使用关键字参数的好处：
1. 使代码更容易理解
2. 可以在函数定义中提供默认值
3. 可以提供一种扩充函数参数的有效方式，使得扩充之后的函数依然能与原有的那些调用代码相兼容，不需要迁移大量代码，即可给函数添加新的功能，这减少了引入bug的概率

# 用None和文档字符串来描述具有动态默认值的参数

1. 参数的默认值，只会在程序加载该模块并读到本函数的定义时评估一次。对于{}或[]等动态的值，这可能会导致奇怪的行为
2. 对于以动态值作为实际默认值的关键字参数来说，应该把形式上的默认值写为None，并在函数的文档字符串里面描述该默认值所对应的实际行为。而动态部分的实现，应该在函数体中。

# 用只能以关键字形式指定的参数来确保代码明晰

1. 关键字参数能够使函数调用的意图更加明确
2. 对于各参数之间很容易混淆的函数，可以声明只能以关键字形式指定的参数，以确保调用者必须通过关键字来指定它们，对于接受多个Boolean标志的函数，更应该这样做
3. 在编写函数时，Python3有明确的语法来定义这种只能以关键字形式指定的参数，即在参数列表中添加一个\*，它标志着位置参数的结束，\*之后的参数都必须是关键字参数